In [3]:
import skimage 
from skimage import exposure, measure, feature, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [66]:
df = pd.read_csv('Daten.csv', ';')

In [73]:
#df = df.sort_values(by=['image_id'])
df.head()

,image_id,dx,validation,malignant,contrast,dissimilarity,homogeneity,energy,correlation,average_red,...,mu11,mu02,mu03,mu21,mu12,nu20,nu30,nu21,nu12,nu03
0,ISIC_0024306,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
1,ISIC_0024307,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
2,ISIC_0024308,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
3,ISIC_0024309,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
4,ISIC_0024310,mel,0,1,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value


In [74]:
df = df[['image_id','dx', 'validation', 'malignant']].copy()

In [75]:
df.head()

,image_id,dx,validation,malignant
0,ISIC_0024306,nv,0,0
1,ISIC_0024307,nv,0,0
2,ISIC_0024308,nv,0,0
3,ISIC_0024309,nv,0,0
4,ISIC_0024310,mel,0,1


In [71]:
df.head()

,image_id,dx,validation,malignant,contrast,dissimilarity,homogeneity,energy,correlation,average_red,...,mu11,mu02,mu03,mu21,mu12,nu20,nu30,nu21,nu12,nu03
0,ISIC_0024306,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
1,ISIC_0024307,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
2,ISIC_0024308,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
3,ISIC_0024309,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
4,ISIC_0024310,mel,0,1,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value


In [37]:
df.loc[df['image_id'] == 'ISIC_00' + '24306', 'dx'].values[0]

'nv'

In [55]:
df.head()

,image_id,dx,validation,malignant,contrast,dissimilarity,homogeneity,energy,correlation,average_red,...,mu11,mu02,mu03,mu21,mu12,nu20,nu30,nu21,nu12,nu03
0,ISIC_0024306,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
1,ISIC_0024307,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
2,ISIC_0024308,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
3,ISIC_0024309,nv,0,0,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value
4,ISIC_0024310,mel,0,1,default value,default value,default value,default value,default value,default value,...,default value,default value,default value,default value,default value,default value,default value,default value,default value,default value


In [52]:
#24306-34320 (-34321)
d = []
for image_id in range(24306,34321):
    
    img = cv2.imread('Intelligent Systems in Medicine/Project/ISIC2018_Task3_Training_Input/ISIC_00' + str(image_id) +'.jpg')
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (17, 17), 32)
    ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    im2, contours,hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key=cv2.contourArea)
    if len(cnt) > 4:
        ellipse = cv2.fitEllipse(cnt)
        x,y,w,h = cv2.boundingRect(cnt)
        area = hsv[int(y+(0.3*h)):int(y+(0.8*h)),int((0.2*w)+x):int((0.7*w)+x)]
        ellipse_cnt = cv2.ellipse2Poly( (int(ellipse[0][0]),int(ellipse[0][1]) ) ,( int(ellipse[1][0]),int(ellipse[1][1]) ),int(ellipse[2]),0,360,1)
      
    
   
    # add average color information to dataframe
    mask = thresh > 0
    red_sum = np.multiply(img[:,:,0],mask).sum()
    green_sum = np.multiply(img[:,:,1],mask).sum()
    blue_sum = np.multiply(img[:,:,2],mask).sum()

    area_sum = mask.sum()

     #TODO: histogram equalization
     # add texture information to dataframe
    gCoMat = feature.greycomatrix(gray,[2],[0], 256, symmetric = True, normed = True)
    
    #color without segmentation
    average = img.mean(axis=0).mean(axis=0)
    
    # add perimeter, symmetry and area variance
    perimeter = cv2.arcLength(cnt,True)
    comp = cv2.matchShapes(cnt,ellipse_cnt,1,0.0)
    variance = cv2.meanStdDev(area)
    
    
    #add moment information to dataframe
    moments = cv2.moments(cnt, True)
    
    d.append({'image_id': 'ISIC_00' + str(image_id),
     'contrast': feature.greycoprops(gCoMat, prop='contrast'),
     'dissimilarity': feature.greycoprops(gCoMat, prop='dissimilarity'),
     'homogeneity': feature.greycoprops(gCoMat, prop='homogeneity'),
     'energy': feature.greycoprops(gCoMat, prop='energy'),
     'correlation': feature.greycoprops(gCoMat, prop='correlation'),
     'average_red': red_sum/area_sum,
     'average_green': green_sum/area_sum,
     'average_blue':blue_sum/area_sum,
     'average_red2': average[0],
     'average_green2': average[1],
     'average_blue2': average[2],
     'perimeter': perimeter,
     'symmetry': comp,
     'area_variance1': variance[1][0][0],
     'area_variance2': variance[1][1][0],
     'area_variance3': variance[1][2][0],
     'area_variance01': variance[0][0][0],
     'area_variance02': variance[0][1][0],
     'area_variance03': variance[0][2][0],
     'area': moments['m00'],
     'm10': moments['m10'],
     'm01': moments['m01'],
     'm20': moments['m20'],
     'm11': moments['m11'],
     'm02': moments['m02'],
     'm30': moments['m30'],
     'm21': moments['m21'],
     'm12': moments['m12'],
     'm03': moments['m03'],
     'mu20': moments['mu20'],
     'mu11': moments['mu11'],
     'mu02': moments['mu02'],
     'mu03': moments['mu03'],
     'mu21': moments['mu21'],
     'mu12': moments['mu12'],
     'mu03': moments['mu03'],
     'nu20': moments['nu20'],
     'nu30': moments['nu30'],
     'nu21': moments['nu21'],
     'nu12': moments['nu12'],
     'nu03': moments['nu03']
    })

    
    

In [76]:
df = pd.DataFrame(d).set_index('image_id').join(df.set_index('image_id'))

In [77]:
df.head()

,area,area_variance01,area_variance02,area_variance03,area_variance1,area_variance2,area_variance3,average_blue,average_blue2,average_green,...,nu03,nu12,nu20,nu21,nu30,perimeter,symmetry,dx,validation,malignant
image_id,,,,,,,,,,,,,,,,,,,,,
ISIC_0024306,55753.0,113.617597,184.429171,143.167927,85.325693,14.157758,29.372298,170.708336,214.742830,60.785949,...,-0.004953,-0.004660,0.066802,0.001744,0.001999,1175.283393,0.056284,nv,0,0
ISIC_0024307,51264.0,60.152981,150.014751,193.760493,83.918039,17.260601,12.629358,197.792446,216.408637,88.609574,...,-0.002422,0.006293,0.087290,0.001290,-0.005850,1348.253955,0.153526,nv,0,0
ISIC_0024308,80633.5,34.955079,181.790820,145.373889,69.106006,31.471933,36.710583,155.571393,211.505348,53.042818,...,0.002476,0.004335,0.140067,0.003432,-0.000241,1347.692555,0.056716,nv,0,0
ISIC_0024309,55425.5,173.797075,171.508705,152.693182,2.037120,13.166800,13.236745,164.196650,193.472681,62.231370,...,-0.002172,-0.002769,0.109403,0.004772,0.004881,1163.844795,0.103694,nv,0,0
ISIC_0024310,42664.5,53.887443,130.269553,158.705142,77.814072,32.021363,37.005539,151.371173,201.125259,76.905045,...,0.004127,0.006592,0.148071,-0.012777,-0.026188,1600.319971,0.204217,mel,0,1


In [78]:
df.to_csv('features_all.csv')